# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization
nltk.download('punkt')


from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

import re
import pickle
import time
from joblib import dump

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
table_name = "DisasterResponse"
df = pd.read_sql_table(table_name,engine)
df.sample(5)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
5307,6034,option : gestion of bussiness,Dajeanson jocelaine. Option gestion des affair...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7892,8868,"Hi, is immigration making new passport in this...",SALUT EST IMMIGRATION EMET DE NOUVEAU PASPORT ...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25692,29666,More than 400 sachets of residual spray have a...,None,news,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
9516,10710,Are there cycline tomorrow Tuesday please,Eske gen cyclone demen madi a s.v.p.?,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21375,24826,Of the 25 sites in total in Northern Sri Lanka...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26215 entries, 0 to 26214
Data columns (total 40 columns):
id                        26215 non-null int64
message                   26215 non-null object
original                  10170 non-null object
genre                     26215 non-null object
related                   26215 non-null int64
request                   26215 non-null int64
offer                     26215 non-null int64
aid_related               26215 non-null int64
medical_help              26215 non-null int64
medical_products          26215 non-null int64
search_and_rescue         26215 non-null int64
security                  26215 non-null int64
military                  26215 non-null int64
child_alone               26215 non-null int64
water                     26215 non-null int64
food                      26215 non-null int64
shelter                   26215 non-null int64
clothing                  26215 non-null int64
money                     26215 non-null i

In [6]:
engine.table_names()

['DisasterResponse']

In [7]:
X = df.iloc[:,1] 
y = df.iloc[:,4:]
category_names = y.columns

In [8]:
y.shape

(26215, 36)

In [9]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    
    """
    inputs:
    messages
       
    Returns:
    >>>> list of words into numbers of same meaning
    """ 
       
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [11]:
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([
            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),
        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 20972 samples.
Testing set has 5243 samples.


In [14]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [15]:
y_preds = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
def evaluate_model(y_test, y_preds, category_names):
    print(classification_report(y_test, y_preds, target_names=category_names))
    print('------------------------------------------------------------------')
    for i in range(y.shape[1]):
        print("Category:", category_names[i],
              "\n", 
              classification_report(y_test.iloc[:, i].values, y_preds[:, i]))
        print('Accuracy: {:,.2f}%'.format(accuracy_score(y_test.iloc[:, i].values, y_preds[:,i])*100),"\n")

In [17]:
evaluate_model(y_test, y_preds, category_names)

                        precision    recall  f1-score   support

               related       0.80      0.98      0.88      4018
               request       0.73      0.52      0.61       898
                 offer       0.00      0.00      0.00        21
           aid_related       0.76      0.60      0.67      2191
          medical_help       0.60      0.24      0.35       422
      medical_products       0.69      0.35      0.47       279
     search_and_rescue       0.55      0.18      0.27       156
              security       0.33      0.03      0.05       102
              military       0.63      0.36      0.46       185
           child_alone       0.00      0.00      0.00         0
                 water       0.73      0.67      0.70       356
                  food       0.80      0.64      0.71       574
               shelter       0.75      0.54      0.63       446
              clothing       0.64      0.40      0.49        73
                 money       0.56      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'text_pipeline', 'classifier', 'text_pipeline__memory', 'text_pipeline__steps', 'text_pipeline__count_vectorizer', 'text_pipeline__tfidf_transformer', 'text_pipeline__count_vectorizer__analyzer', 'text_pipeline__count_vectorizer__binary', 'text_pipeline__count_vectorizer__decode_error', 'text_pipeline__count_vectorizer__dtype', 'text_pipeline__count_vectorizer__encoding', 'text_pipeline__count_vectorizer__input', 'text_pipeline__count_vectorizer__lowercase', 'text_pipeline__count_vectorizer__max_df', 'text_pipeline__count_vectorizer__max_features', 'text_pipeline__count_vectorizer__min_df', 'text_pipeline__count_vectorizer__ngram_range', 'text_pipeline__count_vectorizer__preprocessor', 'text_pipeline__count_vectorizer__stop_words', 'text_pipeline__count_vectorizer__strip_accents', 'text_pipeline__count_vectorizer__token_pattern', 'text_pipeline__count_vectorizer__tokenizer', 'text_pipeline__count_vectorizer__vocabulary', 'text_pipeline__tfidf_transformer__

In [24]:
#parameters = {'text_pipeline__tfidf_transformer__use_idf': [True, False],
#    'classifier__estimator__n_estimators': [10, 20, 40]}

parameters = {'classifier__estimator__n_estimators': [10, 25, 50]}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', n_jobs=-1)

In [ ]:
start= time.time()
cv.fit(X_train, y_train)
print("Durations: {} seconds".format(time.time() - start))

In [46]:
print(cv.best_params_)
print(cv.best_score_)

{'classifier__estimator__n_estimators': 40}
0.658627563109


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [47]:
evaluate_model(y_test, y_preds, category_names)

                        precision    recall  f1-score   support

               related       0.79      0.97      0.88      4036
               request       0.75      0.52      0.61       873
                 offer       0.00      0.00      0.00        25
           aid_related       0.76      0.62      0.69      2159
          medical_help       0.60      0.31      0.41       402
      medical_products       0.62      0.35      0.44       273
     search_and_rescue       0.66      0.24      0.35       137
              security       0.46      0.07      0.12        91
              military       0.58      0.29      0.38       171
           child_alone       0.00      0.00      0.00         0
                 water       0.78      0.63      0.69       355
                  food       0.81      0.67      0.73       556
               shelter       0.78      0.53      0.63       462
              clothing       0.71      0.34      0.46        95
                 money       0.63      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline_new = Pipeline([
            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),
        ('classifier', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [ ]:
pipeline_new.fit(X_train, y_train)

In [ ]:
y_preds_np = pipeline.predict(X_test)

In [ ]:
evaluate_model(y_test, y_preds_np, category_names)

### 9. Export your model as a pickle file

In [56]:
def pickle_model(model, pkl_filename):
    with open(pkl_filename, 'wb') as file:
        pickle.dump(model, file)

In [57]:
pickle_model(pipeline, 'classifier.pkl')

PicklingError: Can't pickle <function tokenize at 0x7fbb20634158>: it's not the same object as __main__.tokenize

In [55]:
#joblib.dump(pipeline, 'classifier.pkl')
#joblib.dump(cv, 'classifier.pkl')

PicklingError: Can't pickle <function tokenize at 0x7fbb20634158>: it's not the same object as __main__.tokenize

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.